In [2]:
import sqlite3

db_path = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Documents\USIU_A\US2025\DSA2040A\Final Exam\DSA_2040_Practical_Exam_Ambachow_550\Data_Warehousing\design\retail_dw.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()


[('CustomerDim',), ('ProductDim',), ('TimeDim',), ('sqlite_sequence',), ('SalesFact',)]


In [3]:
import sqlite3

db_path = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Documents\USIU_A\US2025\DSA2040A\Final Exam\DSA_2040_Practical_Exam_Ambachow_550\Data_Warehousing\design\retail_dw.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(SalesFact);")
columns = cursor.fetchall()
for col in columns:
    print(col)

conn.close()


(0, 'Description', 'TEXT', 0, None, 0)
(1, 'Quantity', 'INTEGER', 0, None, 0)
(2, 'InvoiceDate', 'TIMESTAMP', 0, None, 0)
(3, 'UnitPrice', 'REAL', 0, None, 0)
(4, 'CustomerID', 'TEXT', 0, None, 0)
(5, 'Country', 'TEXT', 0, None, 0)
(6, 'InvoiceYear', 'INTEGER', 0, None, 0)
(7, 'InvoiceMonth', 'INTEGER', 0, None, 0)
(8, 'InvoiceDay', 'INTEGER', 0, None, 0)
(9, 'InvoiceQuarter', 'INTEGER', 0, None, 0)


In [4]:
import sqlite3

db_path = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Documents\USIU_A\US2025\DSA2040A\Final Exam\DSA_2040_Practical_Exam_Ambachow_550\Data_Warehousing\design\retail_dw.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    table = table_name[0]
    print(f"Schema for table: {table}")
    cursor.execute(f"PRAGMA table_info({table});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  {col[1]} ({col[2]})")  # column name and type
    print("\n")

conn.close()


Schema for table: CustomerDim
  CustomerID (INTEGER)
  CustomerName (TEXT)
  Location (TEXT)
  Country (TEXT)


Schema for table: ProductDim
  ProductID (TEXT)
  ProductName (TEXT)
  Category (TEXT)


Schema for table: TimeDim
  TimeID (INTEGER)
  Date (TEXT)
  Month (INTEGER)
  Quarter (INTEGER)
  Year (INTEGER)


Schema for table: sqlite_sequence
  name ()
  seq ()


Schema for table: SalesFact
  Description (TEXT)
  Quantity (INTEGER)
  InvoiceDate (TIMESTAMP)
  UnitPrice (REAL)
  CustomerID (TEXT)
  Country (TEXT)
  InvoiceYear (INTEGER)
  InvoiceMonth (INTEGER)
  InvoiceDay (INTEGER)
  InvoiceQuarter (INTEGER)




In [6]:
import sqlite3

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(SalesFact);")
columns = cursor.fetchall()

print("Schema for table: SalesFact")
for col in columns:
    print(f"  {col[1]} ({col[2]})")  # col[1] = column name, col[2] = type

conn.close()


Schema for table: SalesFact
  Description (TEXT)
  Quantity (INTEGER)
  InvoiceDate (TIMESTAMP)
  UnitPrice (REAL)
  CustomerID (TEXT)
  Country (TEXT)
  InvoiceYear (INTEGER)
  InvoiceMonth (INTEGER)
  InvoiceDay (INTEGER)
  InvoiceQuarter (INTEGER)


In [5]:
import sqlite3
import pandas as pd
import os

# Define paths
db_path = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Documents\USIU_A\US2025\DSA2040A\Final Exam\DSA_2040_Practical_Exam_Ambachow_550\Data_Warehousing\design\retail_dw.db"
reports_dir = r"C:\Users\Admin\OneDrive - United States International University (USIU)\Documents\USIU_A\US2025\DSA2040A\Final Exam\DSA_2040_Practical_Exam_Ambachow_550\Data_Warehousing\reports"

# Make sure reports folder exists
os.makedirs(reports_dir, exist_ok=True)

def run_query_save_csv(query: str, filename: str):
    # Connect to DB
    conn = sqlite3.connect(db_path)
    
    # Run query
    df = pd.read_sql_query(query, conn)
    
    # Save to CSV
    save_path = os.path.join(reports_dir, filename)
    df.to_csv(save_path, index=False)
    
    # Close connection
    conn.close()
    
    print(f"Query results saved to {save_path}")
    print(df.head())  # Optional: print first 5 rows

# Query 1: Roll-up total sales by country and quarter
query1 = """
SELECT c.Country, t.Year, t.Quarter, SUM(sf.SalesAmount) AS TotalSales
FROM SalesFact sf
JOIN CustomerDim c ON sf.CustomerID = c.CustomerID
JOIN TimeDim t ON sf.TimeID = t.TimeID
GROUP BY c.Country, t.Year, t.Quarter
ORDER BY c.Country, t.Year, t.Quarter;
"""
run_query_save_csv(query1, "rollup_sales_by_country_quarter.csv")

# Query 2: Drill-down sales details for UK by month
query2 = """
SELECT t.Year, t.Month, sf.InvoiceNo, p.ProductName, sf.Quantity, sf.SalesAmount
FROM SalesFact sf
JOIN CustomerDim c ON sf.CustomerID = c.CustomerID
JOIN ProductDim p ON sf.ProductID = p.ProductID
JOIN TimeDim t ON sf.TimeID = t.TimeID
WHERE c.Country = 'United Kingdom'
ORDER BY t.Year, t.Month;
"""
run_query_save_csv(query2, "drilldown_sales_UK_by_month.csv")

# Query 3: Slice total sales for Electronics category by month/year
query3 = """
SELECT t.Year, t.Month, SUM(sf.SalesAmount) AS TotalSales
FROM SalesFact sf
JOIN ProductDim p ON sf.ProductID = p.ProductID
JOIN TimeDim t ON sf.TimeID = t.TimeID
WHERE p.Category = 'Electronics'
GROUP BY t.Year, t.Month
ORDER BY t.Year, t.Month;
"""
run_query_save_csv(query3, "slice_sales_electronics.csv")


DatabaseError: Execution failed on sql '
SELECT c.Country, t.Year, t.Quarter, SUM(sf.SalesAmount) AS TotalSales
FROM SalesFact sf
JOIN CustomerDim c ON sf.CustomerID = c.CustomerID
JOIN TimeDim t ON sf.TimeID = t.TimeID
GROUP BY c.Country, t.Year, t.Quarter
ORDER BY c.Country, t.Year, t.Quarter;
': no such column: sf.SalesAmount